In [ ]:
# 231113
# script for github

In [1]:
import pandas as pd
import itertools

In [2]:
# for the comparisons from the original Orthofinder outputs
main_path = r"/Users/irene/Dropbox (Personal)/UCLA/Lab"

In [3]:
# function to count number of genes per species in orthogroups
def countGenes(OGfile, program, startNum):
    
    orthogroup = pd.read_table(OGfile)
    print("orthogroup shape:", orthogroup.shape)
    OGcount = orthogroup.fillna("*")
    
    if program == "orthofinder":
        for row, value in OGcount.iterrows():
            index = OGcount.shape[1]
            
            # if start with Phylogenetically Hieirachical Orthogroups, start with i=3
            # if start with Orthogroups (no downstream analyses), start with i=1
            i=startNum
            while i < index:
                items = value[i]
                #print(orthogroup.iloc[row][i])
                if type(items) != int:
                    geneList = items.split(", ")
                    #print(geneList)
                    if geneList[0] == "*":
                        OGcount.iat[row,i] = 0
                        i += 1
                    else:
                        numOfGenes = len(geneList)
                        OGcount.iat[row,i] = numOfGenes
                        i += 1
                else:
                    i += 1
                    
    elif program == "sonicparanoid":
        
        OGcount.columns = [col_name.split(".")[0] for col_name in OGcount.columns]
        
        for row, value in OGcount.iterrows():
            index = OGcount.shape[1]
            
            i=1
            while i < index:
                items = value[i]
                #print(orthogroup.iloc[row][i])
                if type(items) != int:
                    geneList = items.split(",")
                    #print(geneList)
                    if geneList[0] == "*":
                        OGcount.iat[row,i] = 0
                        i += 1
                    else:
                        numOfGenes = len(geneList)
                        OGcount.iat[row,i] = numOfGenes
                        i += 1
                else:
                    i += 1
                    
    elif program == "broccoli":
        OGcount.columns = [col_name.split(".")[0] for col_name in OGcount.columns]
        
        for row, value in OGcount.iterrows():
            index = OGcount.shape[1]

            i=1
            while i < index:
                items = value[i]
                #print(orthogroup.iloc[row][i])
                if type(items) != int:
                    geneList = items.split(" ")
                    #print(geneList)
                    if geneList[0] == "*":
                        OGcount.iat[row,i] = 0
                        i += 1
                    else:
                        numOfGenes = len(geneList)
                        OGcount.iat[row,i] = numOfGenes
                        i += 1
                else:
                    i += 1
                    
    return(OGcount)
    #OGcount.to_csv(outputPath, sep = "\t", index=False)

In [4]:
# function to count number of ortholog categories
# 1:1 orthologs
# 1:M orthologs
# M:1 orthologs
# M:M orthologs

def orthologCategory(OGdf, species1, species2):
    
    print("orthogroup shape:", OGdf.shape)
    OGdf_pair = OGdf[[species1, species2]]
    
    one2one = 0
    one2many = 0
    many2one = 0
    many2many = 0
    
    for row, value in OGdf_pair.iterrows():
        #print(value)
        #print(value[0])
        #print(value[1])
        sp1Num = value[0]
        sp2Num = value[1]
        
        if sp1Num == 1 and sp2Num == 1:
            one2one += 1
        elif sp1Num == 1 and sp2Num > 1:
            one2many += 1
        elif sp1Num > 1 and sp2Num == 1:
            many2one += 1
        elif sp1Num > 1 and sp2Num > 1:
            many2many += 1
    
    orthologCountList = [species1+species2, one2one, one2many, many2one, many2many]
    return(orthologCountList)

# Table for baseline counts
## from OrthoFinder-BLAST-MCL for each species pair

In [ ]:
# Loop to generate table for baseline counts
# relies on the pair files to have similar path structure

speciesList = ['Ath', 'Chi', 'Cru', 'Tar', 'Sal', 'Bra', 'Csa', 'Aar']
speciesPairMasterList = []
pairs = itertools.combinations(speciesList, 2)
for item in pairs:
    species1 = item[0]
    species2 = item[1]
    pairFile = main_path + "/path_to_all_files/"+species1 + species2+"/Orthogroups/Orthogroups.tsv"
    countDF = countGenes(pairFile, "orthofinder", 1)
    speciesPair = species1+species2+"List"
    speciesPair = orthologCategory(countDF, species1, species2)
    speciesPairMasterList.append(speciesPair)

In [ ]:
speciesPairMasterList

In [7]:
orthoCatDF = pd.DataFrame(speciesPairMasterList, columns = ["SpPair", "one:one", "1:M", "M:1", "M:M"])

In [8]:
output_path = main_path+"/path_to_output/baselineOF_blast_230221.csv"
orthoCatDF.to_csv(output_path, index=False)

# For each orthology inferernce algorithms
## diploid set and higher ploidy set 

In [5]:
# function for orthogroups more than 2 species: test on brass_d 

def orthoCatMasterList(OGfile, program, indNum, speciesList, output_path):
    countDF = countGenes(OGfile, program, indNum)
    
    speciesPairMasterList = []
    
    pairs = itertools.combinations(speciesList, 2)
    for item in pairs:
        species1 = item[0]
        species2 = item[1]
        speciesPair = species1+species2+"List"
        speciesPair = orthologCategory(countDF, species1, species2)
        speciesPairMasterList.append(speciesPair)

    orthoCatDF = pd.DataFrame(speciesPairMasterList, columns = ["SpPair", "one:one", "1:M", "M:1", "M:M"])
    orthoCatDF.to_csv(output_path, index=False)
    

### Broccoli

In [ ]:
# broccoli - diploid
BR_path = main_path + "/path_to_dip/dip_BR-table_OGs_protein_names.txt"
BR_out = main_path + "/path_to_out/dip_BR_orthoCat_230927.csv"
diploidList = ['Ath', 'Chi', 'Cru', 'Tar', 'Aar']
orthoCatMasterList(BR_path, "broccoli", 1, diploidList, BR_out)

In [ ]:
# broccoli - higher ploidy
BR_path = main_path + "/path_to_full/full_BR-table_OGs_protein_names.txt"
BR_out = main_path + "/path_to_out/full_BR_orthoCat_230927.csv"
fullList = ['Ath', 'Chi', 'Cru', 'Tar', 'Sal', 'Bra', 'Csa', 'Aar']
orthoCatMasterList(BR_path, "broccoli", 1, fullList, BR_out)

### OrthoFinder-BLAST

In [ ]:
# Orthofinder-BLAST - diploid
OF_b_path = main_path + "/path_to_dip/dip_OF_b_N0.tsv"
OF_b_out = main_path + "/path_to_out/dip_OF_blast_orthoCat_230927.csv"
diploidList = ['Ath', 'Chi', 'Cru', 'Tar', 'Aar']
orthoCatMasterList(OF_b_path, "orthofinder", 3, diploidList, OF_b_out)

In [ ]:
# Orthofinder-BLAST - higher ploidy
OF_b_path = main_path + "/path_to_full/full_OFb_N0.tsv"
OF_b_out = main_path + "/path_to_out/dip_OF_blast_orthoCat_230927.csv"
fullList = ['Ath', 'Chi', 'Cru', 'Tar', 'Sal', 'Bra', 'Csa', 'Aar']
orthoCatMasterList(OF_b_path, "orthofinder", 3, fullList, OF_b_out)

### OrthoFinder-DIAMOND

In [ ]:
# Orthofinder-DIAMOND - diploid
OF_d_path = main_path + "/path_to_dip/dip_OF_d_N0.tsv"
OF_d_out = main_path + "/path_to_out/dip_OF_diamond_orthoCat_230927.csv"
diploidList = ['Ath', 'Chi', 'Cru', 'Tar','Aar']
orthoCatMasterList(OF_d_path, "orthofinder", 3, diploidList, OF_d_out)

In [ ]:
# Orthofinder-DIAMOND - higher ploidy
OF_d_path = main_path + "/path_to_full/full_OFd_N0.tsv"
OF_d_out = main_path + "/path_to_out/full_OF_diamond_orthoCat_230213.csv"
fullList = ['Ath', 'Chi', 'Cru', 'Tar', 'Sal', 'Bra', 'Csa', 'Aar']
orthoCatMasterList(OF_d_path, "orthofinder", 3, fullList, OF_d_out)

### OrthoFinder-MMseqs2

In [ ]:
# Orthofinder-MMseqs2 - diploid
OF_m_path = main_path + "/path_to_dip/dip_OF_m_N0.tsv"
OF_m_out = main_path + "/path_to_full/dip_OF_mmseqs_orthoCat_230927.csv"
diploidList = ['Ath', 'Chi', 'Cru', 'Tar', 'Aar']
orthoCatMasterList(OF_m_path, "orthofinder", 3, diploidList, OF_m_out)

In [ ]:
# Orthofinder-MMseqs2 - higher ploidy
OF_m_path = main_path + "/path_to_dip/full_OFm_N0.tsv"
OF_m_out = main_path + "/path_to_full/full_OF_mmseqs_orthoCat_230213.csv"
fullList = ['Ath', 'Chi', 'Cru', 'Tar', 'Sal', 'Bra', 'Csa', 'Aar']
orthoCatMasterList(OF_m_path, "orthofinder", 3, fullList, OF_m_out)

### SonicParanoid-DIAMOND

In [ ]:
# SonicParanoid-DIAMOND - diploid
SP_d_path = main_path + "/path_to_dip/dip_SP_d-flat.ortholog_groups.tsv"
SP_d_out = main_path + "/path_to_out/dip_SP_diamond_orthoCat_230927.csv"
diploidList = ['Ath', 'Chi', 'Cru', 'Tar', 'Aar']
orthoCatMasterList(SP_d_path, "sonicparanoid", 1, diploidList, SP_d_out)

In [ ]:
# SonicParanoid-DIAMOND - higher ploidy
SP_d_path = main_path + "/path_to_full/full_SPd_flat.ortholog_groups.tsv"
SP_d_out = main_path + "/path_to_out/full_SP_diamond_orthoCat_230119.csv"
fullList = ['Ath', 'Chi', 'Cru', 'Tar', 'Sal', 'Bra', 'Csa', 'Aar']
orthoCatMasterList(SP_d_path, "sonicparanoid", 1, fullList, SP_d_out)

### SonicParanoid-MMseqs2

In [ ]:
# SonicParanoid-MMseqs2 - diploid
SP_m_path = main_path + "/path_to_dip/dip_SP_m-flat.ortholog_groups.tsv"
SP_m_out = main_path + "/path_to_out/dip_SP_mmseqs_orthoCat_230927.csv"
diploidList = ['Ath', 'Chi', 'Cru', 'Tar', 'Aar']
orthoCatMasterList(SP_m_path, "sonicparanoid", 1, diploidList, SP_m_out)

In [ ]:
# SonicParanoid-MMseqs2 - higher ploidy
SP_m_path = main_path + "/path_to_full/full_SPm-flat.ortholog_groups.tsv"
SP_m_out = main_path + "/path_to_out/full_SP_mmseqs_orthoCat_230119.csv"
fullList = ['Ath', 'Chi', 'Cru', 'Tar', 'Sal', 'Bra', 'Csa', 'Aar']
orthoCatMasterList(SP_m_path, "sonicparanoid", 1, fullList, SP_m_out)

### OrthNet

In [ ]:
# OrthNet - diploid
ON_path = main_path + "/path_to_dip/OrthNet_groups_diploid_230927.txt"
ON_out = main_path + "/path_to_out/dip_ON_orthoCat_230927.csv"
diploidList = ['Ath', 'Chi', 'Cru', 'Tar', 'Aar']
orthoCatMasterList(ON_path, "orthofinder", 1, diploidList, ON_out)

In [ ]:
# OrthNet - higher ploidy
ON_path = main_path + "/path_to_full/OrthNet_groups_full_230915.txt"
ON_out = main_path + "/path_to_out/full_ON_orthoCat_230915.csv"
fullList = ['Ath', 'Chi', 'Cru', 'Tar', 'Sal', 'Bra', 'Csa', 'Aar']
orthoCatMasterList(ON_path, "orthofinder", 1, fullList, ON_out)